<a href="https://colab.research.google.com/github/theandyhuang/mask_api/blob/main/mask_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Task #07 指定閱讀：Python 實作資料爬蟲與資料收集


In [4]:
import requests
import json

# 利用 requests 對 API 來源發送一個請求
url = "https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json"
response = requests.get(url)

# 將請求回應的內容存成一個字串格式
d = response.text

# 將長得像 json 格式的字串解析成字典或列表
data = json.loads(d)
print(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



計算各地區的藥局數量

In [18]:
med_count = {}

# 填入欄位名稱
for d in data["features"]:
    conunty = d["properties"]['address'][:3]
    if conunty not in med_count:
         med_count[conunty] = 0

    # if conunty == "":
    #   print(d["properties"]['address'])
    med_count[conunty] = med_count[conunty] + 1


    

print(med_count)
# {'台北市': 123, '新北市': 456 ...}

{'臺北市': 340, '高雄市': 424, '臺中市': 436, '臺南市': 280, '基隆市': 57, '新竹市': 39, '嘉義市': 67, '新北市': 507, '桃園市': 271, '新竹縣': 48, '宜蘭縣': 76, '苗栗縣': 57, '彰化縣': 181, '南投縣': 67, '雲林縣': 131, '嘉義縣': 84, '屏東縣': 140, '澎湖縣': 11, '花蓮縣': 47, '臺東縣': 23, '金門縣': 6, '連江縣': 1}


計算出每個地區的成人剩餘口罩數量，並且將結果從大到小排列

In [25]:
mask_count = {}

# 填入欄位名稱
for d in data['features']:
    conunty = d["properties"]['address'][:3]
    mask_adult = d["properties"]['mask_adult']
    if conunty not in mask_count:
      mask_count[conunty] = mask_adult
    else:
      mask_count[conunty] = mask_count[conunty] + mask_adult

# print(mask_count.items())
# 將結果從大到小排列
mask_count = dict(sorted(mask_count.items(), key=lambda item: item[1], reverse=True))

print(mask_count)
# {'台北市': 12345, '新北市': 45678 ...}


{'新北市': 677660, '臺中市': 556440, '高雄市': 428940, '臺南市': 408400, '桃園市': 345600, '臺北市': 323660, '彰化縣': 219400, '雲林縣': 199470, '屏東縣': 184250, '苗栗縣': 134180, '嘉義縣': 113180, '嘉義市': 107840, '宜蘭縣': 106900, '南投縣': 91240, '基隆市': 80100, '花蓮縣': 57330, '新竹縣': 53760, '新竹市': 45630, '臺東縣': 28510, '金門縣': 15510, '澎湖縣': 12600, '連江縣': 7930}


## Task #08 作業#01 【實作題】將資料存入資料庫

將計算的「各地區的藥局數量」存入資料庫的 `pharmacies` 表格，欄位包含「地區」、「數量」、「新增時間」

In [30]:
import sqlite3
import datetime

conn = sqlite3.connect('example.db')
c = conn.cursor()

# 新增且清空資料表
c.execute('''CREATE TABLE IF NOT EXISTS pharmacies
             (city text, counts text, createdAt datetime)''')
c.execute('''DELETE FROM pharmacies''')
conn.commit()

# 新增資料(直接把前面執行完的拿過來用)
data = med_count

for d in data.keys():
  city = d
  counts = data[str(d)]
  t = datetime.datetime.now()
  print(f"INSERT INTO stocks VALUES ('{city}', {counts}, '{t}')")
  c.execute(f"INSERT INTO pharmacies VALUES ('{city}', {counts}, '{t}')")
  conn.commit()

# 查詢資料
c.execute("SELECT * FROM pharmacies")
print(c.fetchall())

conn.commit()
conn.close()

INSERT INTO stocks VALUES ('臺北市', 340, '2023-04-13 12:29:20.628879')
INSERT INTO stocks VALUES ('高雄市', 424, '2023-04-13 12:29:20.639548')
INSERT INTO stocks VALUES ('臺中市', 436, '2023-04-13 12:29:20.647953')
INSERT INTO stocks VALUES ('臺南市', 280, '2023-04-13 12:29:20.656844')
INSERT INTO stocks VALUES ('基隆市', 57, '2023-04-13 12:29:20.665006')
INSERT INTO stocks VALUES ('新竹市', 39, '2023-04-13 12:29:20.673246')
INSERT INTO stocks VALUES ('嘉義市', 67, '2023-04-13 12:29:20.682046')
INSERT INTO stocks VALUES ('新北市', 507, '2023-04-13 12:29:20.690699')
INSERT INTO stocks VALUES ('桃園市', 271, '2023-04-13 12:29:20.699479')
INSERT INTO stocks VALUES ('新竹縣', 48, '2023-04-13 12:29:20.708400')
INSERT INTO stocks VALUES ('宜蘭縣', 76, '2023-04-13 12:29:20.720250')
INSERT INTO stocks VALUES ('苗栗縣', 57, '2023-04-13 12:29:20.729314')
INSERT INTO stocks VALUES ('彰化縣', 181, '2023-04-13 12:29:20.737725')
INSERT INTO stocks VALUES ('南投縣', 67, '2023-04-13 12:29:20.746304')
INSERT INTO stocks VALUES ('雲林縣', 131, '2